# Physical Location Factors in Voter Participation and Behavior:  Undervoting in the 2018 Board of Education Contests in Wake County, North Carolina
This Jupyter Notebook is associated with the accompanying report and presentation of the same name.  It is part of the final deliverable for the Applied Data Science Capstone course in the final course of the IBM Data Science Professional certificate program.

To begin, we first set up our environment, import libraries we may be needing for completion of this analysis and read in the first data set we will be using to analyze the data, and learn about possible factors associated with undervoting for the race in question.

In [1]:
# import libraries that may be used in this notebook
import pandas as pd
import numpy as np

import json
import requests
from pandas.io.json import json_normalize

# !conda install -c conda-forge geopy --yes
# from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import seaborn as sns

import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
%matplotlib inline

#!conda install -c conda-forge folium=0.5.0 --yes
import folium # map rendering library

from sklearn import linear_model
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_boston
from sklearn.metrics import mean_squared_error, r2_score

print('Libraries imported.')

Libraries imported.


In [2]:
# read data from file and create a dataframe to work with
wake_df = pd.read_csv('Polling_Places.csv')
wake_df.head()

,objectid,reid,precinct,st_number,st_name,city,zip,latitude,longitude,poll_pl,poll_pl_coded,last_edited_date
0,3712,89722,01-01,700,BROOKS AVE,RALEIGH,27607,35.796864,-78.672269,Brooks Ave Church of Christ,ch,NaN
1,3569,72098,01-02,3313,WADE AVE,RALEIGH,27607,35.800620,-78.679774,Unitarian Universalist Fellowship of Raleigh,ch,NaN
2,3530,90605,01-03,1615,OBERLIN RD,RALEIGH,27608,35.804298,-78.658087,Saint Johns Baptist Church,ch,NaN
3,3570,102613,01-04,2816,OBERLIN RD,RALEIGH,27608,35.813639,-78.656665,Daniels Middle School,ms,NaN
4,3713,76219,01-05,2010,MCDONALD LN,RALEIGH,27608,35.806657,-78.655556,White Memorial Presbyterian Church,ch,NaN


<br />
We examine the data that we will be using and found it needed to be cleaned up a bit.

In [3]:
wake_df[['city','zip']].groupby(['city']).count()

,zip
city,
APEX,15
CARY,28
CREEDMOOR,1
FUQUAY VARINA,4
FUQUAY-VARINA,2
GARNER,9
HOLLY SPRINGS,5
KNIGHTDALE,5
MORRISVILLE,3


In [4]:
wake_df[['city','poll_pl_coded']].groupby(['poll_pl_coded']).count()

,city
poll_pl_coded,
cc,21
ch,76
club,3
cult,2
es,58
fire,9
gov,3
hs,6
lib,2


In [5]:
# remove un-needed columns - objectid, reid, last_edited_date
wake_df.drop(['objectid','reid','last_edited_date'], axis=1, inplace=True)
wake_df.head()

,precinct,st_number,st_name,city,zip,latitude,longitude,poll_pl,poll_pl_coded
0,01-01,700,BROOKS AVE,RALEIGH,27607,35.796864,-78.672269,Brooks Ave Church of Christ,ch
1,01-02,3313,WADE AVE,RALEIGH,27607,35.800620,-78.679774,Unitarian Universalist Fellowship of Raleigh,ch
2,01-03,1615,OBERLIN RD,RALEIGH,27608,35.804298,-78.658087,Saint Johns Baptist Church,ch
3,01-04,2816,OBERLIN RD,RALEIGH,27608,35.813639,-78.656665,Daniels Middle School,ms
4,01-05,2010,MCDONALD LN,RALEIGH,27608,35.806657,-78.655556,White Memorial Presbyterian Church,ch


In [6]:
# clean up city names
# make 'FUQUAY VARINA' into 'FUQUAY-VARINA'
wake_df[(wake_df['city'] == 'FUQUAY VARINA')]

,precinct,st_number,st_name,city,zip,latitude,longitude,poll_pl,poll_pl_coded
124,12-02,820,S MAIN ST,FUQUAY VARINA,27526,35.573165,-78.801115,Fuquay-Varina Community Center,cc
127,12-06,130,N JUDD PKWY NE,FUQUAY VARINA,27526,35.586226,-78.781060,Wake County Southern Regional Center,gov
128,12-07,10225,CHAMBERS RD,FUQUAY VARINA,27603,35.631573,-78.709602,Banks Road Elementary,es
129,12-08,605,BRIDGE ST,FUQUAY VARINA,27526,35.592348,-78.809250,Saint Augusta Missionary Baptist Church,ch


In [7]:
wake_df.loc[wake_df['city'] == 'FUQUAY VARINA',['city']] = 'FUQUAY-VARINA'

In [8]:
# clean up city names
# make 'WAKE COUNTY' into a city, town, or municipality
wake_df[(wake_df['city'] == 'WAKE COUNTY')]

,precinct,st_number,st_name,city,zip,latitude,longitude,poll_pl,poll_pl_coded
174,18-07,6910,FAYETTVILLE RD,WAKE COUNTY,27603,35.68203,-78.68697,Mid-Way Baptist Church,ch


In [9]:
wake_df.loc[wake_df['city'] == 'WAKE COUNTY',['city']] = 'RALEIGH'

In [10]:
# clean up city names
# make 'ZEBULON NC 27597' into 'ZEBULON'
wake_df[(wake_df['city'] == 'ZEBULON NC 27597')]

,precinct,st_number,st_name,city,zip,latitude,longitude,poll_pl,poll_pl_coded
115,09-02,301,S ARENDELL AVE,ZEBULON NC 27597,27597,35.819055,-78.309978,Zebulon Community Center,cc


In [11]:
wake_df.loc[wake_df['city'] == 'ZEBULON NC 27597',['city']] = 'ZEBULON'

<br />
Now we can tell where all the precincts are in Wake County in terms of cities.

In [12]:
wake_df[['city','zip']].groupby(['city']).count()

,zip
city,
APEX,15
CARY,28
CREEDMOOR,1
FUQUAY-VARINA,6
GARNER,9
HOLLY SPRINGS,5
KNIGHTDALE,5
MORRISVILLE,3
RALEIGH,114


<br />
Check for null values.

In [13]:
# look for NaN fields
wake_df.isnull().any()

precinct         False
st_number        False
st_name          False
city             False
zip              False
latitude         False
longitude        False
poll_pl          False
poll_pl_coded    False
dtype: bool

<br />
Code the precinct polling locations to make them human readable.

In [14]:
# combine poll_pl_coded fields as follows...

#   community centers = cc + park = 21 + 3 = 24
wake_df.loc[wake_df['poll_pl_coded'] == 'cc',['poll_pl_coded']] = 'community centers'
wake_df.loc[wake_df['poll_pl_coded'] == 'park',['poll_pl_coded']] = 'community centers'

#   religious = ch = 76
wake_df.loc[wake_df['poll_pl_coded'] == 'ch',['poll_pl_coded']] = 'religious'

#   clubs culture libraries = club + cult + lib = 3 + 2 + 2
wake_df.loc[wake_df['poll_pl_coded'] == 'club',['poll_pl_coded']] = 'clubs culture libraries'
wake_df.loc[wake_df['poll_pl_coded'] == 'cult',['poll_pl_coded']] = 'clubs culture libraries'
wake_df.loc[wake_df['poll_pl_coded'] == 'lib',['poll_pl_coded']] = 'clubs culture libraries'

#   schools = es + ms + hs + sch = 58 + 12 + 6 + 2 = 78
wake_df.loc[wake_df['poll_pl_coded'] == 'es',['poll_pl_coded']] = 'school'
wake_df.loc[wake_df['poll_pl_coded'] == 'ms',['poll_pl_coded']] = 'school'
wake_df.loc[wake_df['poll_pl_coded'] == 'hs',['poll_pl_coded']] = 'school'
wake_df.loc[wake_df['poll_pl_coded'] == 'sch',['poll_pl_coded']] = 'school'

#   safety = fire + pol = 9 + 1 = 10 
wake_df.loc[wake_df['poll_pl_coded'] == 'fire',['poll_pl_coded']] = 'safety'
wake_df.loc[wake_df['poll_pl_coded'] == 'pol',['poll_pl_coded']] = 'safety'

#   government = gov + soc-srv = 3 + 1 
wake_df.loc[wake_df['poll_pl_coded'] == 'gov',['poll_pl_coded']] = 'government'
wake_df.loc[wake_df['poll_pl_coded'] == 'soc-srv',['poll_pl_coded']] = 'government'

#   residential = res-com + ret-com = 5 + 2 = 7
wake_df.loc[wake_df['poll_pl_coded'] == 'res-com',['poll_pl_coded']] = 'residential'
wake_df.loc[wake_df['poll_pl_coded'] == 'ret-com',['poll_pl_coded']] = 'residential'

In [15]:
wake_df[['city','poll_pl_coded']].groupby(['poll_pl_coded']).count()

,city
poll_pl_coded,
clubs culture libraries,7
community centers,24
government,4
religious,76
residential,7
safety,10
school,78


In [16]:
wake_df.head()

,precinct,st_number,st_name,city,zip,latitude,longitude,poll_pl,poll_pl_coded
0,01-01,700,BROOKS AVE,RALEIGH,27607,35.796864,-78.672269,Brooks Ave Church of Christ,religious
1,01-02,3313,WADE AVE,RALEIGH,27607,35.800620,-78.679774,Unitarian Universalist Fellowship of Raleigh,religious
2,01-03,1615,OBERLIN RD,RALEIGH,27608,35.804298,-78.658087,Saint Johns Baptist Church,religious
3,01-04,2816,OBERLIN RD,RALEIGH,27608,35.813639,-78.656665,Daniels Middle School,school
4,01-05,2010,MCDONALD LN,RALEIGH,27608,35.806657,-78.655556,White Memorial Presbyterian Church,religious


## Review of Descriptive Statistics

We then look at statistics associated with undervote, particularly those in the 2018 midterm elections for Wake County, North Carolina.  We correlate them with turnout just to see what the relationship is if any.  A positive moderate relationship.  The more people turn out to vote, the less they vote for Board of Elections.

In [17]:
# read data from file and create a dataframe to work with
boe_df = pd.read_csv('BoE-coded-2018.csv')
boe_df

,Precinct,RegVot,Cast,Cast-Blank,Turnout,BoE-Cast,Undervote
0,01-01,2331,1332,0,0.5714,990,0.256757
1,01-02,2299,1356,0,0.5898,928,0.315634
2,01-03,2087,1547,0,0.7413,977,0.368455
3,01-04,1297,952,0,0.7340,622,0.346639
4,01-05,1125,861,0,0.7653,551,0.360046
...,...,...,...,...,...,...,...
199,20-12,3513,2424,0,0.6900,2035,0.160479
200,20-14,6463,4028,0,0.6232,3545,0.119911
201,20-15,4092,2294,0,0.5606,1862,0.188317
202,20-16,3711,2014,0,0.5427,1557,0.226912


In [18]:
boe_df.corr(method="pearson")

,RegVot,Cast,Cast-Blank,Turnout,BoE-Cast,Undervote
RegVot,1.000000,0.941927,0.244122,-0.261962,0.899997,-0.243409
Cast,0.941927,1.000000,0.232104,0.036314,0.941248,-0.185581
Cast-Blank,0.244122,0.232104,1.000000,-0.041675,0.226829,-0.070433
Turnout,-0.261962,0.036314,-0.041675,1.000000,-0.019946,0.256715
BoE-Cast,0.899997,0.941248,0.226829,-0.019946,1.000000,-0.481578
Undervote,-0.243409,-0.185581,-0.070433,0.256715,-0.481578,1.000000


In [19]:
boe_df.describe()

,RegVot,Cast,Cast-Blank,Turnout,BoE-Cast,Undervote
count,204.000000,204.000000,204.000000,204.000000,204.000000,204.000000
mean,3607.607843,2152.710784,0.303922,0.607501,1570.411765,0.280034
std,1642.337936,951.778729,0.632288,0.090905,797.315611,0.116823
min,336.000000,173.000000,0.000000,0.266200,111.000000,0.118347
25%,2473.500000,1459.000000,0.000000,0.556600,987.750000,0.170929
50%,3263.000000,1999.500000,0.000000,0.616000,1440.500000,0.265193
75%,4589.000000,2694.250000,0.000000,0.671525,1955.000000,0.385662
max,9267.000000,5336.000000,3.000000,0.811800,4462.000000,0.483589


Undervoting for Board of Elections is as high as 48% in some precincts!

## Onehot Encoding for Types of Polling Places

We code type of polling location for later analysis.  We also join it with our undervote data in the previous section.

In [20]:
polling_df = wake_df.copy(deep=True)

In [21]:
# remove un-needed columns
polling_df.drop(['st_number','st_name','city','zip','latitude','longitude','poll_pl'], axis=1, inplace=True)
polling_df

,precinct,poll_pl_coded
0,01-01,religious
1,01-02,religious
2,01-03,religious
3,01-04,school
4,01-05,religious
...,...,...
201,20-12,school
202,20-14,religious
203,20-15,school
204,20-16,school


In [22]:
# onehot encoding
polling_encoded = pd.get_dummies(polling_df[['poll_pl_coded']], prefix="", prefix_sep="")
polling_encoded

,clubs culture libraries,community centers,government,religious,residential,safety,school
0,0,0,0,1,0,0,0
1,0,0,0,1,0,0,0
2,0,0,0,1,0,0,0
3,0,0,0,0,0,0,1
4,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...
201,0,0,0,0,0,0,1
202,0,0,0,1,0,0,0
203,0,0,0,0,0,0,1
204,0,0,0,0,0,0,1


In [23]:
# add back precinct column
polling_encoded['Precinct'] = wake_df['precinct'] 
n_col = polling_encoded.pop('Precinct')
polling_encoded.insert(0, 'Precinct', n_col)
polling_encoded

,Precinct,clubs culture libraries,community centers,government,religious,residential,safety,school
0,01-01,0,0,0,1,0,0,0
1,01-02,0,0,0,1,0,0,0
2,01-03,0,0,0,1,0,0,0
3,01-04,0,0,0,0,0,0,1
4,01-05,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...
201,20-12,0,0,0,0,0,0,1
202,20-14,0,0,0,1,0,0,0
203,20-15,0,0,0,0,0,0,1
204,20-16,0,0,0,0,0,0,1


In [24]:
boe_polling_encoded_merged = polling_encoded.copy(deep=True)
boe_polling_encoded_merged = boe_df.join(polling_encoded.set_index('Precinct'), on='Precinct')
boe_polling_encoded_merged

,Precinct,RegVot,Cast,Cast-Blank,Turnout,BoE-Cast,Undervote,clubs culture libraries,community centers,government,religious,residential,safety,school
0,01-01,2331,1332,0,0.5714,990,0.256757,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1,01-02,2299,1356,0,0.5898,928,0.315634,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2,01-03,2087,1547,0,0.7413,977,0.368455,0.0,0.0,0.0,1.0,0.0,0.0,0.0
3,01-04,1297,952,0,0.7340,622,0.346639,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,01-05,1125,861,0,0.7653,551,0.360046,0.0,0.0,0.0,1.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
199,20-12,3513,2424,0,0.6900,2035,0.160479,0.0,0.0,0.0,0.0,0.0,0.0,1.0
200,20-14,6463,4028,0,0.6232,3545,0.119911,0.0,0.0,0.0,1.0,0.0,0.0,0.0
201,20-15,4092,2294,0,0.5606,1862,0.188317,0.0,0.0,0.0,0.0,0.0,0.0,1.0
202,20-16,3711,2014,0,0.5427,1557,0.226912,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [25]:
boe_polling_encoded_merged.drop(['Precinct','RegVot','Cast','Cast-Blank','Turnout','BoE-Cast'], axis=1, inplace=True)
boe_polling_encoded_merged

,Undervote,clubs culture libraries,community centers,government,religious,residential,safety,school
0,0.256757,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1,0.315634,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2,0.368455,0.0,0.0,0.0,1.0,0.0,0.0,0.0
3,0.346639,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,0.360046,0.0,0.0,0.0,1.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...
199,0.160479,0.0,0.0,0.0,0.0,0.0,0.0,1.0
200,0.119911,0.0,0.0,0.0,1.0,0.0,0.0,0.0
201,0.188317,0.0,0.0,0.0,0.0,0.0,0.0,1.0
202,0.226912,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [26]:
boe_polling_encoded_merged.corr(method="pearson")

,Undervote,clubs culture libraries,community centers,government,religious,residential,safety,school
Undervote,1.000000,0.008912,-0.129821,-0.060250,0.091947,0.068271,0.044864,-0.038482
clubs culture libraries,0.008912,1.000000,-0.067916,-0.026930,-0.145600,-0.035897,-0.043240,-0.147148
community centers,-0.129821,-0.067916,1.000000,-0.050949,-0.275465,-0.067916,-0.081806,-0.278393
government,-0.060250,-0.026930,-0.050949,1.000000,-0.109226,-0.026930,-0.032437,-0.110387
religious,0.091947,-0.145600,-0.275465,-0.109226,1.000000,-0.145600,-0.175379,-0.596830
residential,0.068271,-0.035897,-0.067916,-0.026930,-0.145600,1.000000,-0.043240,-0.147148
safety,0.044864,-0.043240,-0.081806,-0.032437,-0.175379,-0.043240,1.000000,-0.177244
school,-0.038482,-0.147148,-0.278393,-0.110387,-0.596830,-0.147148,-0.177244,1.000000




## Onehot Encoding of Precinct Venues Gained from the Foursquare API

We set up the Foursquare API and begin to collect venues from around precinct locations.  Precinct locations are identified by latitude and longitude coordinates.  This includes analyzing the frequency of venues and joining our dummy coding with the previously reviewed undervote data.

In [27]:
#pass Foursquare API information
CLIENT_ID = 'HRLKE3GPSUVHVI3DLKALDZWLI4ETYFDP53C2ANXALGXOVD4C' 
CLIENT_SECRET = 'BTMW2I0EBOSAM4I0GUMDT303F5J1I324KRK5N02X24DJLOBZ' 
VERSION = '20180605' 

LIMIT = 100;

print('Foursquare API credentionals stored in CLIENT_ID, CLIENT_SECRET, and VERSION.  LIMIT set to 100.')

Foursquare API credentionals stored in CLIENT_ID, CLIENT_SECRET, and VERSION.  LIMIT set to 100.


In [28]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Precinct', 
                  'Polling Latitude', 
                  'Polling Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [29]:
wake_venues = getNearbyVenues(names=wake_df['precinct'], latitudes=wake_df['latitude'], longitudes=wake_df['longitude'])
print('Venues loaded.')

01-01
01-02
01-03
01-04
01-05
01-06
01-07
01-07A
01-09
01-10
01-11
01-12
01-13
01-14
01-15
01-16
01-17
01-18
01-19
01-20
01-21
01-22
01-23
01-25
01-26
01-27
01-28
01-29
01-30
01-31
01-32
01-33
01-34
01-35
01-36
01-37
01-38
01-39
01-40
01-41
01-42
01-43
01-44
01-45
01-46
01-47
01-48
01-49
01-50
01-51
02-01
02-02
02-03
02-04
02-05
02-06
03-00
04-01
04-02
04-03
04-04
04-05
04-06
04-07
04-08
04-09
04-10
04-11
04-12
04-13
04-14
04-15
04-16
04-17
04-18
04-19
04-20
04-21
05-01
05-03
05-05
05-06
05-07
05-08
06-04
06-05
06-06
06-07
06-08
06-09
06-10
07-01
07-02
07-03
07-04
07-05
07-06
07-07
07-07A
07-09
07-10
07-11
07-12
07-13
08-02
08-03
08-04
08-05
08-06
08-07
08-08
08-09
08-10
08-11
09-01
09-02
09-03
10-01
10-02
10-03
10-04
11-01
11-02
12-01
12-02
12-04
12-05
12-06
12-07
12-08
12-09
13-01
13-02
13-05
13-06
13-07
13-08
13-09
13-10
13-11
14-01
14-02
15-01
15-02
15-03
15-04
16-01
16-02
16-03
16-04
16-05
16-06
16-07
16-09
16-10
16-11
17-01
17-02
17-03
17-04
17-05
17-06
17-07
17-09
17-10
17-11
17

In [30]:
wake_venues.shape

(1340, 7)

In [31]:
wake_venues

,Precinct,Polling Latitude,Polling Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,01-01,35.796864,-78.672269,Zen Garden,35.795030,-78.674992,Garden
1,01-01,35.796864,-78.672269,Trampin,35.796949,-78.676711,Playground
2,01-01,35.796864,-78.672269,Frankenstorm,35.796739,-78.677382,Ski Area
3,01-02,35.800620,-78.679774,Bruegger's Bagels,35.801544,-78.684683,Bagel Shop
4,01-02,35.800620,-78.679774,Papa John's Pizza,35.799941,-78.683463,Pizza Place
...,...,...,...,...,...,...,...
1335,20-15,35.859172,-78.899589,vegetable parcel,35.862039,-78.895689,IT Services
1336,20-16,35.810876,-78.896805,Mills Park Soccer Field,35.812371,-78.895294,Soccer Field
1337,20-16,35.810876,-78.896805,Mills Park Middle School Softball/Baseball Field,35.813849,-78.896259,Baseball Field
1338,20-17,35.800753,-78.884113,Sears Farm Road Park,35.798274,-78.882952,Park


In [32]:
wake_venues.groupby('Precinct').count()

,Polling Latitude,Polling Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Precinct,,,,,,
01-01,3,3,3,3,3,3
01-02,5,5,5,5,5,5
01-03,2,2,2,2,2,2
01-04,10,10,10,10,10,10
01-05,5,5,5,5,5,5
...,...,...,...,...,...,...
20-11,2,2,2,2,2,2
20-12,11,11,11,11,11,11
20-15,5,5,5,5,5,5


In [33]:
# number of unique categories
print('There are {} uniques categories of venues.'.format(len(wake_venues['Venue Category'].unique())))

There are 245 uniques categories of venues.


In [34]:
# onehot encoding
wake_encoded = pd.get_dummies(wake_venues[['Venue Category']], prefix="", prefix_sep="")
wake_encoded.head()

,ATM,Accessories Store,Adult Boutique,Airport,American Restaurant,Antique Shop,Arcade,Art Gallery,Arts & Crafts Store,Asian Restaurant,...,Video Game Store,Video Store,Vietnamese Restaurant,Weight Loss Center,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [35]:
# add back precinct column
wake_encoded['Precinct'] = wake_venues['Precinct'] 
n_col = wake_encoded.pop('Precinct')
wake_encoded.insert(0, 'Precinct', n_col)

In [36]:
wake_encoded

,Precinct,ATM,Accessories Store,Adult Boutique,Airport,American Restaurant,Antique Shop,Arcade,Art Gallery,Arts & Crafts Store,...,Video Game Store,Video Store,Vietnamese Restaurant,Weight Loss Center,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,01-01,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,01-01,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,01-01,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,01-02,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,01-02,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1335,20-15,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1336,20-16,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1337,20-16,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1338,20-17,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [37]:
# mean venue per category by neighborhood
wake_grouped = wake_encoded.groupby('Precinct').mean().reset_index()
wake_grouped

,Precinct,ATM,Accessories Store,Adult Boutique,Airport,American Restaurant,Antique Shop,Arcade,Art Gallery,Arts & Crafts Store,...,Video Game Store,Video Store,Vietnamese Restaurant,Weight Loss Center,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,01-01,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,01-02,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,01-03,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,01-04,0.0,0.0,0.0,0.0,0.100000,0.0,0.0,0.0,0.0,...,0.0,0.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,01-05,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
181,20-11,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
182,20-12,0.0,0.0,0.0,0.0,0.090909,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
183,20-15,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
184,20-16,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [38]:
# function to sort venues into different categories
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [39]:
# new df and top ten venues per negighborhood
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Precinct']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
wake_venues_sorted = pd.DataFrame(columns=columns)
wake_venues_sorted['Precinct'] = wake_grouped['Precinct']

for ind in np.arange(wake_grouped.shape[0]):
    wake_venues_sorted.iloc[ind, 1:] = return_most_common_venues(wake_grouped.iloc[ind, :], num_top_venues)

wake_venues_sorted

,Precinct,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,01-01,Ski Area,Garden,Playground,Yoga Studio,Furniture / Home Store,Frozen Yogurt Shop,Fried Chicken Joint,French Restaurant,Football Stadium,Food Truck
1,01-02,Dry Cleaner,Pizza Place,Bagel Shop,Tailor Shop,Martial Arts Dojo,Food & Drink Shop,Food Court,Food Service,Food Truck,Football Stadium
2,01-03,Fast Food Restaurant,Home Service,Fish Market,Garden Center,Garden,Furniture / Home Store,Frozen Yogurt Shop,Fried Chicken Joint,French Restaurant,Football Stadium
3,01-04,Dessert Shop,Pizza Place,Gym,Tennis Court,Sushi Restaurant,Video Store,Social Club,American Restaurant,Supermarket,Spa
4,01-05,Southern / Soul Food Restaurant,Pizza Place,Salon / Barbershop,Gym,Chinese Restaurant,Food,Food & Drink Shop,Food Court,Food Service,Yoga Studio
...,...,...,...,...,...,...,...,...,...,...,...
181,20-11,Gift Shop,Sporting Goods Shop,Yoga Studio,Flower Shop,Garden,Furniture / Home Store,Frozen Yogurt Shop,Fried Chicken Joint,French Restaurant,Football Stadium
182,20-12,Intersection,Massage Studio,Salon / Barbershop,Playground,Pizza Place,Coffee Shop,Shopping Plaza,Fast Food Restaurant,Grocery Store,Gas Station
183,20-15,Gym,Convenience Store,IT Services,Business Service,Food & Drink Shop,Food Court,Food Service,Food Truck,Football Stadium,Yoga Studio
184,20-16,Soccer Field,Baseball Field,Yoga Studio,Food,Garden Center,Garden,Furniture / Home Store,Frozen Yogurt Shop,Fried Chicken Joint,French Restaurant




## Cluster Analysis of Foursquare API Venues by Precinct

We do some cluster analysis to understand the venue clusters associated with each precinct.

In [40]:
knums = 3
wake_grouped_clustering_k03 = wake_grouped.drop('Precinct', 1)
kclusts = KMeans(n_clusters=knums, random_state=0).fit(wake_grouped_clustering_k03)
kclusts.labels_[0:10].astype(int)

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [41]:
knums = 4
wake_grouped_clustering_k04 = wake_grouped.drop('Precinct', 1)
kclusts = KMeans(n_clusters=knums, random_state=0).fit(wake_grouped_clustering_k04)
kclusts.labels_[0:10].astype(int)

array([2, 2, 2, 2, 2, 2, 2, 2, 2, 2])

In [42]:
knums = 5
wake_grouped_clustering_k05 = wake_grouped.drop('Precinct', 1)
kclusts = KMeans(n_clusters=knums, random_state=0).fit(wake_grouped_clustering_k05)
kclusts.labels_[0:10].astype(int)

array([4, 4, 4, 4, 4, 4, 4, 4, 4, 4])

In [43]:
knums = 6
wake_grouped_clustering_k06 = wake_grouped.drop('Precinct', 1)
kclusts = KMeans(n_clusters=knums, random_state=0).fit(wake_grouped_clustering_k06)
kclusts.labels_[0:10].astype(int)

array([0, 2, 0, 2, 2, 2, 2, 2, 2, 2])

In [44]:
knums = 7
wake_grouped_clustering_k07 = wake_grouped.drop('Precinct', 1)
kclusts = KMeans(n_clusters=knums, random_state=0).fit(wake_grouped_clustering_k07)
kclusts.labels_[0:10].astype(int)

array([4, 5, 5, 5, 5, 5, 5, 5, 5, 5])

In [45]:
knums = 8
wake_grouped_clustering_k08 = wake_grouped.drop('Precinct', 1)
kclusts = KMeans(n_clusters=knums, random_state=0).fit(wake_grouped_clustering_k08)
kclusts.labels_[0:10].astype(int)

array([4, 4, 4, 4, 4, 4, 4, 4, 4, 4])

In [46]:
knums = 9
wake_grouped_clustering_k09 = wake_grouped.drop('Precinct', 1)
kclusts = KMeans(n_clusters=knums, random_state=0).fit(wake_grouped_clustering_k09)
kclusts.labels_[0:10].astype(int)

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [47]:
knums = 10
wake_grouped_clustering_k10 = wake_grouped.drop('Precinct', 1)
kclusts = KMeans(n_clusters=knums, random_state=0).fit(wake_grouped_clustering_k10)
kclusts.labels_[0:10].astype(int)

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

In [48]:
knums = 11
wake_grouped_clustering_k11 = wake_grouped.drop('Precinct', 1)
kclusts = KMeans(n_clusters=knums, random_state=0).fit(wake_grouped_clustering_k11)
kclusts.labels_[0:10].astype(int)

array([2, 1, 2, 1, 1, 2, 2, 2, 1, 2])

In [49]:
knums = 12
wake_grouped_clustering_k12 = wake_grouped.drop('Precinct', 1)
kclusts = KMeans(n_clusters=knums, random_state=0).fit(wake_grouped_clustering_k12)
kclusts.labels_[0:10].astype(int)

array([10, 10,  0, 10, 10, 10, 10, 10, 10, 10])

In [50]:
knums = 13
wake_grouped_clustering_k13 = wake_grouped.drop('Precinct', 1)
kclusts = KMeans(n_clusters=knums, random_state=0).fit(wake_grouped_clustering_k13)
kclusts.labels_[0:10].astype(int)

array([8, 8, 8, 8, 8, 8, 8, 8, 8, 8])

In [51]:
knums = 14
wake_grouped_clustering_k14 = wake_grouped.drop('Precinct', 1)
kclusts = KMeans(n_clusters=knums, random_state=0).fit(wake_grouped_clustering_k14)
kclusts.labels_[0:10].astype(int)

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

In [52]:
knums = 15
wake_grouped_clustering_k15 = wake_grouped.drop('Precinct', 1)
kclusts = KMeans(n_clusters=knums, random_state=0).fit(wake_grouped_clustering_k15)
kclusts.labels_[0:10].astype(int)

array([5, 5, 5, 5, 0, 5, 5, 5, 5, 5])

<br />
Based on cluster results, we chose to evaluate six clusters.

In [54]:
wake_grouped_clustering = wake_grouped_clustering_k06

In [55]:
knums = 6
wake_grouped_clustering = wake_grouped.drop('Precinct', 1)
kclusts = KMeans(n_clusters=knums, random_state=0).fit(wake_grouped_clustering)
kclusts.labels_[0:10].astype(int)

array([0, 2, 0, 2, 2, 2, 2, 2, 2, 2])

In [56]:
# create dataframe with all necessary columns and information
wake_venues_sorted.insert(0, 'Cluster Labels', kclusts.labels_)
wake_merged = wake_df
wake_merged = wake_merged.join(wake_venues_sorted.set_index('Precinct'), on='precinct')
wake_merged

,precinct,st_number,st_name,city,zip,latitude,longitude,poll_pl,poll_pl_coded,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,01-01,700,BROOKS AVE,RALEIGH,27607,35.796864,-78.672269,Brooks Ave Church of Christ,religious,0.0,Ski Area,Garden,Playground,Yoga Studio,Furniture / Home Store,Frozen Yogurt Shop,Fried Chicken Joint,French Restaurant,Football Stadium,Food Truck
1,01-02,3313,WADE AVE,RALEIGH,27607,35.800620,-78.679774,Unitarian Universalist Fellowship of Raleigh,religious,2.0,Dry Cleaner,Pizza Place,Bagel Shop,Tailor Shop,Martial Arts Dojo,Food & Drink Shop,Food Court,Food Service,Food Truck,Football Stadium
2,01-03,1615,OBERLIN RD,RALEIGH,27608,35.804298,-78.658087,Saint Johns Baptist Church,religious,0.0,Fast Food Restaurant,Home Service,Fish Market,Garden Center,Garden,Furniture / Home Store,Frozen Yogurt Shop,Fried Chicken Joint,French Restaurant,Football Stadium
3,01-04,2816,OBERLIN RD,RALEIGH,27608,35.813639,-78.656665,Daniels Middle School,school,2.0,Dessert Shop,Pizza Place,Gym,Tennis Court,Sushi Restaurant,Video Store,Social Club,American Restaurant,Supermarket,Spa
4,01-05,2010,MCDONALD LN,RALEIGH,27608,35.806657,-78.655556,White Memorial Presbyterian Church,religious,2.0,Southern / Soul Food Restaurant,Pizza Place,Salon / Barbershop,Gym,Chinese Restaurant,Food,Food & Drink Shop,Food Court,Food Service,Yoga Studio
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
201,20-12,1751,OLIVE CHAPEL RD,APEX,27502,35.734786,-78.894563,Olive Chapel Elementary School,school,0.0,Intersection,Massage Studio,Salon / Barbershop,Playground,Pizza Place,Coffee Shop,Shopping Plaza,Fast Food Restaurant,Grocery Store,Gas Station
202,20-14,1621,WHITE OAK CHURCH RD,APEX,27523,35.780153,-78.918912,White Oak Missionary Baptist Church,religious,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
203,20-15,11555,GREEN LEVEL CHURCH RD,CARY,27519,35.859172,-78.899589,ALSTON RIDGE ELEMENTARY SCHOOL,school,2.0,Gym,Convenience Store,IT Services,Business Service,Food & Drink Shop,Food Court,Food Service,Food Truck,Football Stadium,Yoga Studio
204,20-16,509,MILLS PARK DR,CARY,27519,35.810876,-78.896805,Mills Park Elementary School,school,4.0,Soccer Field,Baseball Field,Yoga Studio,Food,Garden Center,Garden,Furniture / Home Store,Frozen Yogurt Shop,Fried Chicken Joint,French Restaurant


In [57]:
# NaN on the category for precincts with no nearby venues
# write neighborhoods with no venues to a separate dataframe 
wake_no_venues = wake_merged[wake_merged["Cluster Labels"].isnull()]
wake_no_venues

,precinct,st_number,st_name,city,zip,latitude,longitude,poll_pl,poll_pl_coded,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
41,01-43,5801,FALLS OF NEUSE RD,RALEIGH,27609,35.859677,-78.619098,Saint Raphael Catholic Church,religious,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
46,01-48,4900,KAPLAN DR,RALEIGH,27606,35.778246,-78.705044,Western Boulevard Presbyterian Church,religious,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
51,02-02,1900,PLEASANT UNION CHURCH RD,RALEIGH,27614,35.950439,-78.673435,Pleasant Union Elementary School,school,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
53,02-04,2001,BRASSFIELD RD,RALEIGH,27614,35.933536,-78.630916,Brassfield Elementary School,school,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
65,04-09,1000,OLDE WEATHERSTONE WAY,CARY,27513,35.792303,-78.810193,Weatherstone Elementary School,school,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
66,04-10,2450,LAURA DUNCAN RD,APEX,27523,35.759048,-78.826201,Laurel Park Elementary School,school,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
96,07-06,8301,OLD LEAD MINE RD,RALEIGH,27615,35.891397,-78.657222,Lead Mine Elementary School,school,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
112,08-10,8410,PRIDE WAY,RALEIGH,27613,35.890997,-78.718612,Leesville Road High School,school,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
120,10-04,11109,POOLE RD,WENDELL,27591,35.767467,-78.409640,Central Baptist Church,religious,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
129,12-08,605,BRIDGE ST,FUQUAY-VARINA,27526,35.592348,-78.809250,Saint Augusta Missionary Baptist Church,religious,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [58]:
wake_merged_count = wake_merged.groupby('Cluster Labels').count()
wake_merged_count['precinct']

Cluster Labels
0.0     22
1.0      9
2.0    131
3.0      3
4.0     10
5.0     11
Name: precinct, dtype: int64

In [59]:
wake_merged.loc[wake_merged['Cluster Labels'] == 0, wake_merged.columns[[0,8] + list(range(9, wake_merged.shape[1]))]]

,precinct,poll_pl_coded,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,01-01,religious,0.0,Ski Area,Garden,Playground,Yoga Studio,Furniture / Home Store,Frozen Yogurt Shop,Fried Chicken Joint,French Restaurant,Football Stadium,Food Truck
2,01-03,religious,0.0,Fast Food Restaurant,Home Service,Fish Market,Garden Center,Garden,Furniture / Home Store,Frozen Yogurt Shop,Fried Chicken Joint,French Restaurant,Football Stadium
16,01-17,religious,0.0,Bank,Basketball Court,Fast Food Restaurant,Yoga Studio,Gas Station,Garden Center,Garden,Furniture / Home Store,Frozen Yogurt Shop,Fried Chicken Joint
20,01-21,community centers,0.0,Dog Run,Playground,Park,Yoga Studio,Flower Shop,Garden,Furniture / Home Store,Frozen Yogurt Shop,Fried Chicken Joint,French Restaurant
32,01-34,community centers,0.0,Fast Food Restaurant,Hardware Store,Mattress Store,Wine Bar,Fried Chicken Joint,Automotive Shop,Park,Lawyer,Food Court,Food
43,01-45,religious,0.0,Italian Restaurant,Fast Food Restaurant,Harbor / Marina,Tanning Salon,Pizza Place,Food Truck,Food & Drink Shop,Food Court,Food Service,Football Stadium
44,01-46,community centers,0.0,Indie Movie Theater,Playground,Park,Yoga Studio,Flower Shop,Garden,Furniture / Home Store,Frozen Yogurt Shop,Fried Chicken Joint,French Restaurant
49,01-51,residential,0.0,Massage Studio,Disc Golf,Playground,Yoga Studio,Garden Center,Garden,Furniture / Home Store,Frozen Yogurt Shop,Fried Chicken Joint,French Restaurant
75,04-19,religious,0.0,Intersection,Arcade,Flower Shop,Gas Station,Garden Center,Garden,Furniture / Home Store,Frozen Yogurt Shop,Fried Chicken Joint,French Restaurant
80,05-05,community centers,0.0,Gas Station,Pool,Gym,Pharmacy,Pizza Place,Playground,Boutique,French Restaurant,Football Stadium,Flower Shop


In [60]:
wake_merged.loc[wake_merged['Cluster Labels'] == 1, wake_merged.columns[[0,8] + list(range(9, wake_merged.shape[1]))]]

,precinct,poll_pl_coded,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
27,01-29,school,1.0,Hunting Supply,Pool,Yoga Studio,Flower Shop,Garden Center,Garden,Furniture / Home Store,Frozen Yogurt Shop,Fried Chicken Joint,French Restaurant
31,01-33,school,1.0,Pool,Campground,Trail,Yoga Studio,Flower Shop,Garden,Furniture / Home Store,Frozen Yogurt Shop,Fried Chicken Joint,French Restaurant
40,01-42,community centers,1.0,Pool,Park,Tennis Court,Track Stadium,Dog Run,Flower Shop,Food,Food & Drink Shop,Food Court,Food Service
85,06-05,school,1.0,Pool,Athletics & Sports,Soccer Field,Yoga Studio,Flower Shop,Garden,Furniture / Home Store,Frozen Yogurt Shop,Fried Chicken Joint,French Restaurant
109,08-07,clubs culture libraries,1.0,Pool,Yoga Studio,Flower Shop,Garden Center,Garden,Furniture / Home Store,Frozen Yogurt Shop,Fried Chicken Joint,French Restaurant,Football Stadium
137,13-09,school,1.0,Pool,Playground,Yoga Studio,Fish Market,Garden,Furniture / Home Store,Frozen Yogurt Shop,Fried Chicken Joint,French Restaurant,Football Stadium
145,15-04,school,1.0,ATM,Pool,Farm,Flower Shop,Garden Center,Garden,Furniture / Home Store,Frozen Yogurt Shop,Fried Chicken Joint,French Restaurant
187,19-18,school,1.0,Pool,Yoga Studio,Flower Shop,Garden Center,Garden,Furniture / Home Store,Frozen Yogurt Shop,Fried Chicken Joint,French Restaurant,Football Stadium
205,20-17,school,1.0,Pool,Park,Yoga Studio,Flower Shop,Garden,Furniture / Home Store,Frozen Yogurt Shop,Fried Chicken Joint,French Restaurant,Football Stadium


In [61]:
wake_merged.loc[wake_merged['Cluster Labels'] == 2, wake_merged.columns[[0,8] + list(range(9, wake_merged.shape[1]))]]

,precinct,poll_pl_coded,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,01-02,religious,2.0,Dry Cleaner,Pizza Place,Bagel Shop,Tailor Shop,Martial Arts Dojo,Food & Drink Shop,Food Court,Food Service,Food Truck,Football Stadium
3,01-04,school,2.0,Dessert Shop,Pizza Place,Gym,Tennis Court,Sushi Restaurant,Video Store,Social Club,American Restaurant,Supermarket,Spa
4,01-05,religious,2.0,Southern / Soul Food Restaurant,Pizza Place,Salon / Barbershop,Gym,Chinese Restaurant,Food,Food & Drink Shop,Food Court,Food Service,Yoga Studio
5,01-06,clubs culture libraries,2.0,Boutique,Furniture / Home Store,Jewelry Store,Spa,Yoga Studio,Mexican Restaurant,Coffee Shop,Mediterranean Restaurant,Breakfast Spot,Sporting Goods Shop
6,01-07,school,2.0,Sushi Restaurant,Bar,Arts & Crafts Store,Comedy Club,Mexican Restaurant,Historic Site,Brewery,Cupcake Shop,Hot Dog Joint,Beer Store
...,...,...,...,...,...,...,...,...,...,...,...,...,...
197,20-08,religious,2.0,Music Store,Food Truck,Coffee Shop,Baseball Field,Park,Food,Garden Center,Garden,Furniture / Home Store,Frozen Yogurt Shop
198,20-09,religious,2.0,Mexican Restaurant,Sandwich Place,Supermarket,Martial Arts Dojo,Fast Food Restaurant,Furniture / Home Store,Supplement Shop,Convenience Store,Cosmetics Shop,Chinese Restaurant
199,20-10,safety,2.0,Pizza Place,Ice Cream Shop,Coffee Shop,Fast Food Restaurant,Pet Store,Doctor's Office,Gift Shop,Martial Arts Dojo,Supermarket,Spa
200,20-11,religious,2.0,Gift Shop,Sporting Goods Shop,Yoga Studio,Flower Shop,Garden,Furniture / Home Store,Frozen Yogurt Shop,Fried Chicken Joint,French Restaurant,Football Stadium


In [62]:
wake_merged.loc[wake_merged['Cluster Labels'] == 3, wake_merged.columns[[0,8] + list(range(9, wake_merged.shape[1]))]]

,precinct,poll_pl_coded,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
61,04-05,school,3.0,Garden Center,Athletics & Sports,Yoga Studio,Flower Shop,Gas Station,Garden,Furniture / Home Store,Frozen Yogurt Shop,Fried Chicken Joint,French Restaurant
188,19-19,school,3.0,Gym Pool,Athletics & Sports,Yoga Studio,Flower Shop,Garden Center,Garden,Furniture / Home Store,Frozen Yogurt Shop,Fried Chicken Joint,French Restaurant
189,19-20,community centers,3.0,Athletics & Sports,Yoga Studio,Flower Shop,Gas Station,Garden Center,Garden,Furniture / Home Store,Frozen Yogurt Shop,Fried Chicken Joint,French Restaurant


In [63]:
wake_merged.loc[wake_merged['Cluster Labels'] == 4, wake_merged.columns[[0,8] + list(range(9, wake_merged.shape[1]))]]

,precinct,poll_pl_coded,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
12,01-13,religious,4.0,Baseball Field,Park,Bus Stop,Spa,Gas Station,Garden,Furniture / Home Store,Frozen Yogurt Shop,Fried Chicken Joint,French Restaurant
30,01-32,community centers,4.0,Baseball Field,Yoga Studio,Food,Gas Station,Garden Center,Garden,Furniture / Home Store,Frozen Yogurt Shop,Fried Chicken Joint,French Restaurant
119,10-03,school,4.0,Park,American Restaurant,Baseball Field,Yoga Studio,Gas Station,Garden Center,Garden,Furniture / Home Store,Frozen Yogurt Shop,Fried Chicken Joint
123,12-01,school,4.0,Pool,Farm,Baseball Field,Yoga Studio,Food,Garden Center,Garden,Furniture / Home Store,Frozen Yogurt Shop,Fried Chicken Joint
124,12-02,community centers,4.0,Baseball Field,Playground,Discount Store,Convenience Store,Yoga Studio,Food Court,Food Service,Food Truck,Football Stadium,Fried Chicken Joint
147,16-02,school,4.0,Baseball Field,Yoga Studio,Food,Gas Station,Garden Center,Garden,Furniture / Home Store,Frozen Yogurt Shop,Fried Chicken Joint,French Restaurant
178,19-07,community centers,4.0,Dog Run,Baseball Field,Yoga Studio,Food,Gas Station,Garden Center,Garden,Furniture / Home Store,Frozen Yogurt Shop,Fried Chicken Joint
182,19-13,school,4.0,Football Stadium,Baseball Field,Yoga Studio,Food,Gas Station,Garden Center,Garden,Furniture / Home Store,Frozen Yogurt Shop,Fried Chicken Joint
193,20-04,school,4.0,Baseball Field,Martial Arts Dojo,Construction & Landscaping,Food,Gas Station,Garden Center,Garden,Furniture / Home Store,Frozen Yogurt Shop,Fried Chicken Joint
204,20-16,school,4.0,Soccer Field,Baseball Field,Yoga Studio,Food,Garden Center,Garden,Furniture / Home Store,Frozen Yogurt Shop,Fried Chicken Joint,French Restaurant


In [64]:
wake_merged.loc[wake_merged['Cluster Labels'] == 5, wake_merged.columns[[0,8] + list(range(9, wake_merged.shape[1]))]]

,precinct,poll_pl_coded,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
14,01-15,school,5.0,Night Market,Park,Yoga Studio,Gas Station,Garden,Furniture / Home Store,Frozen Yogurt Shop,Fried Chicken Joint,French Restaurant,Football Stadium
15,01-16,community centers,5.0,Moving Target,Park,Yoga Studio,Food,Garden Center,Garden,Furniture / Home Store,Frozen Yogurt Shop,Fried Chicken Joint,French Restaurant
24,01-26,community centers,5.0,Park,Food Service,Health & Beauty Service,Yoga Studio,Food,Garden Center,Garden,Furniture / Home Store,Frozen Yogurt Shop,Fried Chicken Joint
36,01-38,school,5.0,Park,Yoga Studio,Gastropub,Garden Center,Garden,Furniture / Home Store,Frozen Yogurt Shop,Fried Chicken Joint,French Restaurant,Football Stadium
38,01-40,community centers,5.0,Insurance Office,Park,Yoga Studio,Flower Shop,Garden Center,Garden,Furniture / Home Store,Frozen Yogurt Shop,Fried Chicken Joint,French Restaurant
50,02-01,school,5.0,Park,Yoga Studio,Gastropub,Garden Center,Garden,Furniture / Home Store,Frozen Yogurt Shop,Fried Chicken Joint,French Restaurant,Football Stadium
73,04-17,safety,5.0,Bus Station,Park,Yoga Studio,Food,Garden Center,Garden,Furniture / Home Store,Frozen Yogurt Shop,Fried Chicken Joint,French Restaurant
94,07-04,school,5.0,Miscellaneous Shop,Park,Yoga Studio,Food,Garden Center,Garden,Furniture / Home Store,Frozen Yogurt Shop,Fried Chicken Joint,French Restaurant
157,17-02,school,5.0,Boxing Gym,Park,Yoga Studio,Food,Garden Center,Garden,Furniture / Home Store,Frozen Yogurt Shop,Fried Chicken Joint,French Restaurant
162,17-07,government,5.0,Park,Yoga Studio,Gastropub,Garden Center,Garden,Furniture / Home Store,Frozen Yogurt Shop,Fried Chicken Joint,French Restaurant,Football Stadium


In [65]:
# NaN on the category for precincts with no nearby venues
wake_no_venues = wake_merged[wake_merged["Cluster Labels"].isnull()]
wake_no_venues

,precinct,st_number,st_name,city,zip,latitude,longitude,poll_pl,poll_pl_coded,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
41,01-43,5801,FALLS OF NEUSE RD,RALEIGH,27609,35.859677,-78.619098,Saint Raphael Catholic Church,religious,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
46,01-48,4900,KAPLAN DR,RALEIGH,27606,35.778246,-78.705044,Western Boulevard Presbyterian Church,religious,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
51,02-02,1900,PLEASANT UNION CHURCH RD,RALEIGH,27614,35.950439,-78.673435,Pleasant Union Elementary School,school,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
53,02-04,2001,BRASSFIELD RD,RALEIGH,27614,35.933536,-78.630916,Brassfield Elementary School,school,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
65,04-09,1000,OLDE WEATHERSTONE WAY,CARY,27513,35.792303,-78.810193,Weatherstone Elementary School,school,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
66,04-10,2450,LAURA DUNCAN RD,APEX,27523,35.759048,-78.826201,Laurel Park Elementary School,school,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
96,07-06,8301,OLD LEAD MINE RD,RALEIGH,27615,35.891397,-78.657222,Lead Mine Elementary School,school,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
112,08-10,8410,PRIDE WAY,RALEIGH,27613,35.890997,-78.718612,Leesville Road High School,school,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
120,10-04,11109,POOLE RD,WENDELL,27591,35.767467,-78.409640,Central Baptist Church,religious,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
129,12-08,605,BRIDGE ST,FUQUAY-VARINA,27526,35.592348,-78.809250,Saint Augusta Missionary Baptist Church,religious,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN




## Regression Analysis of Types of Polling Places

We look at various factors associated with type of polling place and undervote.  There are only weak relationships...  and schools are the weakest relationship with undervoting.  Community centers are the strongest weak relationship against undervoting while religious buildings are the strongest weak relationship in favor of undervoting.

In [66]:
boe_polling_encoded_merged

,Undervote,clubs culture libraries,community centers,government,religious,residential,safety,school
0,0.256757,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1,0.315634,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2,0.368455,0.0,0.0,0.0,1.0,0.0,0.0,0.0
3,0.346639,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,0.360046,0.0,0.0,0.0,1.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...
199,0.160479,0.0,0.0,0.0,0.0,0.0,0.0,1.0
200,0.119911,0.0,0.0,0.0,1.0,0.0,0.0,0.0
201,0.188317,0.0,0.0,0.0,0.0,0.0,0.0,1.0
202,0.226912,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [67]:
# clean up
boe_polling_encoded_merged = boe_polling_encoded_merged.dropna(axis=0)
boe_polling_encoded_merged.reset_index(drop=True)
boe_polling_encoded_merged

,Undervote,clubs culture libraries,community centers,government,religious,residential,safety,school
0,0.256757,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1,0.315634,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2,0.368455,0.0,0.0,0.0,1.0,0.0,0.0,0.0
3,0.346639,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,0.360046,0.0,0.0,0.0,1.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...
199,0.160479,0.0,0.0,0.0,0.0,0.0,0.0,1.0
200,0.119911,0.0,0.0,0.0,1.0,0.0,0.0,0.0
201,0.188317,0.0,0.0,0.0,0.0,0.0,0.0,1.0
202,0.226912,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [68]:
boe_polling_encoded_merged['clubs culture libraries'] = boe_polling_encoded_merged['clubs culture libraries'].astype(int)
boe_polling_encoded_merged['government'] = boe_polling_encoded_merged['government'].astype(int)
boe_polling_encoded_merged['religious'] = boe_polling_encoded_merged['religious'].astype(int)
boe_polling_encoded_merged['residential'] = boe_polling_encoded_merged['residential'].astype(int)
boe_polling_encoded_merged['community centers'] = boe_polling_encoded_merged['community centers'].astype(int)
boe_polling_encoded_merged['safety'] = boe_polling_encoded_merged['safety'].astype(int)
boe_polling_encoded_merged['school'] = boe_polling_encoded_merged['school'].astype(int)

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[

In [69]:
lm_school = linear_model.LinearRegression()
train_x_school = np.asanyarray(boe_polling_encoded_merged[['school']])
train_y_school = np.asanyarray(boe_polling_encoded_merged[['Undervote']])
lm_school.fit (train_x_school, train_y_school)
# The coefficients
print ('Coefficients: ', lm_school.coef_)
print ('Intercept: ',lm_school.intercept_)

Coefficients:  [[-0.00927986]]
Intercept:  [0.28405473]


In [70]:
lm_church = linear_model.LinearRegression()
train_x_church = np.asanyarray(boe_polling_encoded_merged[['religious']])
train_y_church = np.asanyarray(boe_polling_encoded_merged[['Undervote']])
lm_church.fit (train_x_church, train_y_church)
# The coefficients
print ('Coefficients: ', lm_church.coef_)
print ('Intercept: ',lm_church.intercept_)

Coefficients:  [[0.02223227]]
Intercept:  [0.27230874]


In [71]:
lm_cc = linear_model.LinearRegression()
train_x_cc = np.asanyarray(boe_polling_encoded_merged[['community centers']])
train_y_cc = np.asanyarray(boe_polling_encoded_merged[['Undervote']])
lm_cc.fit (train_x_cc, train_y_cc)
# The coefficients
print ('Coefficients: ', lm_cc.coef_)
print ('Intercept: ',lm_cc.intercept_)

Coefficients:  [[-0.04774574]]
Intercept:  [0.28599969]




## Regression Analysis of Precinct Venues

We looked at relationships of venues nearest to polling locations with regard to undervoting.  Relationships were weak, although some venues produced a stronger weak relationship in favor of undervoting.

In [72]:
boe_polling_wake_grouped = wake_grouped.copy(deep=True)
boe_polling_wake_grouped = boe_df.join(wake_grouped.set_index('Precinct'), on='Precinct')
boe_polling_wake_grouped

,Precinct,RegVot,Cast,Cast-Blank,Turnout,BoE-Cast,Undervote,ATM,Accessories Store,Adult Boutique,...,Video Game Store,Video Store,Vietnamese Restaurant,Weight Loss Center,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,01-01,2331,1332,0,0.5714,990,0.256757,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,01-02,2299,1356,0,0.5898,928,0.315634,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,01-03,2087,1547,0,0.7413,977,0.368455,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,01-04,1297,952,0,0.7340,622,0.346639,0.0,0.0,0.0,...,0.0,0.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,01-05,1125,861,0,0.7653,551,0.360046,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
199,20-12,3513,2424,0,0.6900,2035,0.160479,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
200,20-14,6463,4028,0,0.6232,3545,0.119911,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
201,20-15,4092,2294,0,0.5606,1862,0.188317,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
202,20-16,3711,2014,0,0.5427,1557,0.226912,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [73]:
# clean up data prior to correlation/regression
boe_polling_wake_grouped.drop(['Precinct','RegVot','Cast','Cast-Blank','Turnout','BoE-Cast'], axis=1, inplace=True)
boe_polling_wake_grouped = boe_polling_wake_grouped.dropna(axis=0)
boe_polling_wake_grouped.reset_index(drop=True)
boe_polling_wake_grouped

,Undervote,ATM,Accessories Store,Adult Boutique,Airport,American Restaurant,Antique Shop,Arcade,Art Gallery,Arts & Crafts Store,...,Video Game Store,Video Store,Vietnamese Restaurant,Weight Loss Center,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,0.256757,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.315634,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.368455,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.346639,0.0,0.0,0.0,0.0,0.100000,0.0,0.0,0.0,0.0,...,0.0,0.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.360046,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
198,0.148703,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
199,0.160479,0.0,0.0,0.0,0.0,0.090909,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
201,0.188317,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
202,0.226912,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [74]:
four_corr = boe_polling_wake_grouped.corr(method="pearson")
pd.options.display.max_columns = 500
four_corr

,Undervote,ATM,Accessories Store,Adult Boutique,Airport,American Restaurant,Antique Shop,Arcade,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Automotive Shop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Court,Basketball Stadium,Beer Bar,Beer Garden,Beer Store,Big Box Store,Bookstore,Boutique,Bowling Alley,Boxing Gym,Breakfast Spot,Brewery,Bridal Shop,Buffet,Building,Burger Joint,Bus Station,Bus Stop,Business Service,Butcher,Café,Campground,Candy Store,Caribbean Restaurant,Carpet Store,Chinese Restaurant,Circus,Clothing Store,Club House,Cocktail Bar,Coffee Shop,Comedy Club,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Cupcake Shop,Cycle Studio,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Disc Golf,Discount Store,Dive Bar,Doctor's Office,Dog Run,Donut Shop,Drugstore,Dry Cleaner,Electronics Store,Farm,Farmers Market,Fast Food Restaurant,Fish Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Service,Food Truck,Football Stadium,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Garden,Garden Center,Gas Station,Gastropub,General College & University,Gift Shop,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Harbor / Marina,Hardware Store,Health & Beauty Service,Himalayan Restaurant,Historic Site,History Museum,Home Service,Hookah Bar,Hot Dog Joint,Hotel,Hotel Pool,Hotpot Restaurant,Housing Development,Hunting Supply,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Insurance Office,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Kids Store,Korean Restaurant,Lake,Latin American Restaurant,Laundromat,Lawyer,Lingerie Store,Liquor Store,Locksmith,Lounge,Market,Martial Arts Dojo,Massage Studio,Mattress Store,Medical Supply Store,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Miscellaneous Shop,Mobile Phone Shop,Mongolian Restaurant,Motel,Motorcycle Shop,Movie Theater,Moving Target,Music Store,Music Venue,Nail Salon,Neighborhood,New American Restaurant,Night Market,Nightclub,Nightlife Spot,North Indian Restaurant,Office,Optical Shop,Other Repair Shop,Outlet Store,Paper / Office Supplies Store,Park,Performing Arts Venue,Peruvian Restaurant,Pet Store,Pharmacy,Photography Studio,Pizza Place,Playground,Pool,Pool Hall,Print Shop,Pub,Record Shop,Rental Car Location,Residential Building (Apartment / Condo),Resort,Rest Area,Restaurant,Rock Club,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,School,Seafood Restaurant,Shipping Store,Shoe Store,Shop & Service,Shopping Mall,Shopping Plaza,Skate Park,Skating Rink,Ski Area,Ski Shop,Smoke Shop,Snack Place,Soccer Field,Social Club,South Indian Restaurant,Southern / Soul Food Restaurant,Spa,Sporting Goods Shop,Sports Bar,Stables,Stadium,Steakhouse,Supermarket,Supplement Shop,Surf Spot,Sushi Restaurant,Taco Place,Tailor Shop,Tanning Salon,Tapas Restaurant,Tennis Court,Tex-Mex Restaurant,Thai Restaurant,Theater,Theme Park Ride / Attraction,Thrift / Vintage Store,Tour Provider,Tourist Information Center,Toy / Game Store,Track,Track Stadium,Trail,Train Station,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Weight Loss Center,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
Undervote,1.000000,0.085132,-0.020593,0.045983,0.065360,-0.083146,-0.017600,0.123019,-0.085239,0.021688,-0.079468,0.044345,-0.004569,-0.067079,0.044328,-0.006168,-0.015993,0.074724,-0.075735,-0.066538,0.006574,-0.084394,-0.072860,0.011212,0.000592,0.060846,-0.124002,0.009438,-0.062268,-0.066243,0.116209,-0.043570,-0.093227,0.075658,-0.083991,0.128821,0.093943,0.090350,0.007830,-0.079638,-0.079784,0.028220,-0.101513,0.027040,-0.048709,0.104399,0.100004,0.092712,0.067575,0.011212,-0.018546,0.030642,0.122370,-0.052728,-0.078698,-0.049180,0.022035,0.030642,0.022828,-0.091748,-0.038172,0.077185,

In [75]:
four_corr.loc[four_corr['Undervote'] < -0.1]

,Undervote,ATM,Accessories Store,Adult Boutique,Airport,American Restaurant,Antique Shop,Arcade,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Automotive Shop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Court,Basketball Stadium,Beer Bar,Beer Garden,Beer Store,Big Box Store,Bookstore,Boutique,Bowling Alley,Boxing Gym,Breakfast Spot,Brewery,Bridal Shop,Buffet,Building,Burger Joint,Bus Station,Bus Stop,Business Service,Butcher,Café,Campground,Candy Store,Caribbean Restaurant,Carpet Store,Chinese Restaurant,Circus,Clothing Store,Club House,Cocktail Bar,Coffee Shop,Comedy Club,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Cupcake Shop,Cycle Studio,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Disc Golf,Discount Store,Dive Bar,Doctor's Office,Dog Run,Donut Shop,Drugstore,Dry Cleaner,Electronics Store,Farm,Farmers Market,Fast Food Restaurant,Fish Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Service,Food Truck,Football Stadium,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Garden,Garden Center,Gas Station,Gastropub,General College & University,Gift Shop,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Harbor / Marina,Hardware Store,Health & Beauty Service,Himalayan Restaurant,Historic Site,History Museum,Home Service,Hookah Bar,Hot Dog Joint,Hotel,Hotel Pool,Hotpot Restaurant,Housing Development,Hunting Supply,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Insurance Office,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Kids Store,Korean Restaurant,Lake,Latin American Restaurant,Laundromat,Lawyer,Lingerie Store,Liquor Store,Locksmith,Lounge,Market,Martial Arts Dojo,Massage Studio,Mattress Store,Medical Supply Store,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Miscellaneous Shop,Mobile Phone Shop,Mongolian Restaurant,Motel,Motorcycle Shop,Movie Theater,Moving Target,Music Store,Music Venue,Nail Salon,Neighborhood,New American Restaurant,Night Market,Nightclub,Nightlife Spot,North Indian Restaurant,Office,Optical Shop,Other Repair Shop,Outlet Store,Paper / Office Supplies Store,Park,Performing Arts Venue,Peruvian Restaurant,Pet Store,Pharmacy,Photography Studio,Pizza Place,Playground,Pool,Pool Hall,Print Shop,Pub,Record Shop,Rental Car Location,Residential Building (Apartment / Condo),Resort,Rest Area,Restaurant,Rock Club,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,School,Seafood Restaurant,Shipping Store,Shoe Store,Shop & Service,Shopping Mall,Shopping Plaza,Skate Park,Skating Rink,Ski Area,Ski Shop,Smoke Shop,Snack Place,Soccer Field,Social Club,South Indian Restaurant,Southern / Soul Food Restaurant,Spa,Sporting Goods Shop,Sports Bar,Stables,Stadium,Steakhouse,Supermarket,Supplement Shop,Surf Spot,Sushi Restaurant,Taco Place,Tailor Shop,Tanning Salon,Tapas Restaurant,Tennis Court,Tex-Mex Restaurant,Thai Restaurant,Theater,Theme Park Ride / Attraction,Thrift / Vintage Store,Tour Provider,Tourist Information Center,Toy / Game Store,Track,Track Stadium,Trail,Train Station,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Weight Loss Center,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
Bookstore,-0.124002,-0.002834,-0.013386,-0.014787,-0.011160,0.061092,-0.015380,-0.011160,0.012899,-0.013821,-0.018240,-0.023373,0.022318,-0.020462,-0.011160,0.184143,0.027731,-0.017205,0.081561,-0.041230,-0.025418,-0.011160,-0.012668,0.277355,0.224534,-0.015417,1.000000,0.058972,-0.011160,-0.011160,-0.016356,-0.018150,0.016369,0.137441,-0.012989,-0.019917,-0.018191,-0.015349,-0.018806,-0.011160,0.110436,-0.015349,0.188476,-0.011160,-0.011160,-0.015006,-0.011160,-0.011273,-0.011160,0.277355,-0.026773,-0.011160,0.098108,-0.011160,-0.031154,-0.015264,0.081405,-0.011160,-0.011160,-0.018851,0.45

<br />
Parks and shopping malls near polling locations had the strongest relationship against undervoting.

In [76]:
four_corr.loc[four_corr['Undervote'] > 0.1]

,Undervote,ATM,Accessories Store,Adult Boutique,Airport,American Restaurant,Antique Shop,Arcade,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Automotive Shop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Court,Basketball Stadium,Beer Bar,Beer Garden,Beer Store,Big Box Store,Bookstore,Boutique,Bowling Alley,Boxing Gym,Breakfast Spot,Brewery,Bridal Shop,Buffet,Building,Burger Joint,Bus Station,Bus Stop,Business Service,Butcher,Café,Campground,Candy Store,Caribbean Restaurant,Carpet Store,Chinese Restaurant,Circus,Clothing Store,Club House,Cocktail Bar,Coffee Shop,Comedy Club,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Cupcake Shop,Cycle Studio,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Disc Golf,Discount Store,Dive Bar,Doctor's Office,Dog Run,Donut Shop,Drugstore,Dry Cleaner,Electronics Store,Farm,Farmers Market,Fast Food Restaurant,Fish Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Service,Food Truck,Football Stadium,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Garden,Garden Center,Gas Station,Gastropub,General College & University,Gift Shop,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Harbor / Marina,Hardware Store,Health & Beauty Service,Himalayan Restaurant,Historic Site,History Museum,Home Service,Hookah Bar,Hot Dog Joint,Hotel,Hotel Pool,Hotpot Restaurant,Housing Development,Hunting Supply,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Insurance Office,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Kids Store,Korean Restaurant,Lake,Latin American Restaurant,Laundromat,Lawyer,Lingerie Store,Liquor Store,Locksmith,Lounge,Market,Martial Arts Dojo,Massage Studio,Mattress Store,Medical Supply Store,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Miscellaneous Shop,Mobile Phone Shop,Mongolian Restaurant,Motel,Motorcycle Shop,Movie Theater,Moving Target,Music Store,Music Venue,Nail Salon,Neighborhood,New American Restaurant,Night Market,Nightclub,Nightlife Spot,North Indian Restaurant,Office,Optical Shop,Other Repair Shop,Outlet Store,Paper / Office Supplies Store,Park,Performing Arts Venue,Peruvian Restaurant,Pet Store,Pharmacy,Photography Studio,Pizza Place,Playground,Pool,Pool Hall,Print Shop,Pub,Record Shop,Rental Car Location,Residential Building (Apartment / Condo),Resort,Rest Area,Restaurant,Rock Club,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,School,Seafood Restaurant,Shipping Store,Shoe Store,Shop & Service,Shopping Mall,Shopping Plaza,Skate Park,Skating Rink,Ski Area,Ski Shop,Smoke Shop,Snack Place,Soccer Field,Social Club,South Indian Restaurant,Southern / Soul Food Restaurant,Spa,Sporting Goods Shop,Sports Bar,Stables,Stadium,Steakhouse,Supermarket,Supplement Shop,Surf Spot,Sushi Restaurant,Taco Place,Tailor Shop,Tanning Salon,Tapas Restaurant,Tennis Court,Tex-Mex Restaurant,Thai Restaurant,Theater,Theme Park Ride / Attraction,Thrift / Vintage Store,Tour Provider,Tourist Information Center,Toy / Game Store,Track,Track Stadium,Trail,Train Station,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Weight Loss Center,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
Undervote,1.000000,0.085132,-0.020593,0.045983,0.065360,-0.083146,-0.017600,0.123019,-0.085239,0.021688,-0.079468,0.044345,-0.004569,-0.067079,0.044328,-0.006168,-0.015993,0.074724,-0.075735,-0.066538,0.006574,-0.084394,-0.072860,0.011212,0.000592,0.060846,-0.124002,0.009438,-0.062268,-0.066243,0.116209,-0.043570,-0.093227,0.075658,-0.083991,0.128821,0.093943,0.090350,0.007830,-0.079638,-0.079784,0.028220,-0.101513,0.027040,-0.048709,0.104399,0.100004,0.092712,0.067575,0.011212,-0.018546,0.030642,0.122370,-0.052728,-0.078698,-0.049180,0.022035,0.030642,0.022828,-0.091748,-0.038172,0.077185,

<br />
Toy and game stores had the strongest weak relationship with regard to increasing undervoting.  Oddly, farms and garden shops were high as well.  Many types of entertainment businesses/venues made this list of relationships.